In [1]:
from general_tools.notebook import gpu_utils
gpu_utils.setup_one_gpu(0)

Picking GPU 0


In [2]:
import sys
import time
import numpy as np
import os.path as osp
import tensorflow as tf
import matplotlib.pyplot as plt

from tf_lab.fundamentals.utils import reset_tf_graph

import tf_lab.point_clouds.in_out as pio
from tf_lab.point_clouds.in_out import PointCloudDataSet, write_model_ids_of_datasets
from tf_lab.point_clouds.point_net_ae import PointNetAutoEncoder
from tf_lab.point_clouds.autoencoder import Configuration as Conf
import tf_lab.point_clouds.encoders_decoders as enc_dec

from tf_lab.autopredictors.scripts.helper import shape_net_category_to_synth_id, match_incomplete_to_complete_data

from tf_lab.autopredictors.plotting import plot_original_pclouds_vs_reconstructed, \
                                           plot_train_val_test_curves, plot_reconstructions_at_epoch, \
                                           plot_interpolations

from tf_lab.autopredictors.evaluate import eval_model, read_saved_epochs
                                                  

from general_tools.in_out.basics import create_dir, delete_files_in_directory, files_in_subdirs
from general_tools.simpletons import select_first_last_and_k
from geo_tool import Point_Cloud

from tf_lab.nips.shape_net import pc_loader as sn_pc_loader
from tf_lab.nips.eric_semantic_parts import part_pc_loader

/usr/local/lib/python2.7/dist-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')
/orions4-zfs/projects/lins2/Panos_Space/Git_Repos/geo_tool/solids/mesh.py:26: UserWarning: Mayavi library was not found. Some graphics utilities will be disabled.
  warnings.warn('Mayavi library was not found. Some graphics utilities will be disabled.')


In [3]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [4]:
n_pc_samples = 2048
do_training = True
first_time_running = True
load_model = False
seed = 42
max_training_epochs = 1000
loss = 'chamfer'
experiment_name = '1_chair_parts_' + str(n_pc_samples) +  'pts_' + loss

In [5]:
top_data_dir = '/orions4-zfs/projects/lins2/Panos_Space/DATA'
part_pclouds_path = osp.join(top_data_dir, 'Point_Clouds/Shape_Net/With_Part_Annotation_Panos_Version/parts_equisampled/'+str(n_pc_samples)+'_bootstrapped/03001627')
train_dir = osp.join(top_data_dir, 'OUT/models/nips/vanilla_ae/parts')
train_dir = osp.join(train_dir, experiment_name)
create_dir(train_dir)

'/orions4-zfs/projects/lins2/Panos_Space/DATA/OUT/models/nips/vanilla_ae/parts/1_chair_parts_2048pts_chamfer'

In [6]:
part_file_names = pio.load_filenames_of_input_data(part_pclouds_path, '.ply')

In [7]:
part_pclouds, part_model_names, part_syn_ids = pio.load_crude_point_clouds(file_names=part_file_names,\
                                                                           n_threads=40, loader=part_pc_loader)
print '%d files containing parts of full point clouds were found.' % (len(part_pclouds), )

12155 files containing parts of full point clouds were found.


In [8]:
model_names = np.array([i[0] for i in part_model_names], dtype=object)
part_ids = np.array([i[1] for i in part_model_names], dtype=object)
model_unames = np.array(model_names + '.' + part_ids + '.' + part_syn_ids, dtype=object)
train_data = PointCloudDataSet(part_pclouds, labels=model_unames)

In [9]:
if load_model:
    conf = Conf.load(osp.join(train_dir, 'configuration'))
    print conf
else:
    n_input = [n_pc_samples, 3]

    decoder_args = {'layer_sizes': [1024, np.prod(n_input)], 
                    'non_linearity': tf.nn.relu
                   }

    conf = Conf(
                n_input = n_input,
                training_epochs = max_training_epochs,
                batch_size = 50,
                loss = loss,
                denoising = False,
                train_dir = train_dir,
                loss_display_step = 1,
                saver_step = 10,
                learning_rate = 0.0005,
                encoder = enc_dec.encoder_with_convs_and_symmetry,
                decoder = enc_dec.decoder_with_fc_only,
                decoder_args = decoder_args,
               )

    conf.experiment_name = experiment_name
    conf.save(osp.join(conf.train_dir, 'configuration'))
    

reset_tf_graph()
ae = PointNetAutoEncoder(experiment_name, conf)

if load_model:
    saved_epochs = read_saved_epochs(conf.train_dir)
    last_epoch = saved_epochs[-1]
    ae.restore_model(conf.train_dir, last_epoch, verbose=True)

In [10]:
if do_training:
    training_stats = []
    training_stats.append(ae.train(train_data, conf))    
    with open(osp.join(conf.train_dir, 'train_stats.txt'), 'a') as fout:
        np.savetxt(fout, np.array(training_stats)[0])

('Epoch:', '0001', 'training time (minutes)=', '0.3126', 'loss=', '0.003003182')
INFO:tensorflow:/orions4-zfs/projects/lins2/Panos_Space/DATA/OUT/models/nips/vanilla_ae/parts/1_chair_parts_2048pts_chamfer/models.ckpt-1 is not in all_model_checkpoint_paths. Manually adding it.
('Epoch:', '0002', 'training time (minutes)=', '0.3075', 'loss=', '0.001252004')
('Epoch:', '0003', 'training time (minutes)=', '0.3085', 'loss=', '0.001008109')
('Epoch:', '0004', 'training time (minutes)=', '0.3092', 'loss=', '0.000869286')
('Epoch:', '0005', 'training time (minutes)=', '0.3098', 'loss=', '0.000804608')
('Epoch:', '0006', 'training time (minutes)=', '0.3103', 'loss=', '0.000710948')
('Epoch:', '0007', 'training time (minutes)=', '0.3110', 'loss=', '0.000680907')
('Epoch:', '0008', 'training time (minutes)=', '0.3110', 'loss=', '0.000661088')
('Epoch:', '0009', 'training time (minutes)=', '0.3131', 'loss=', '0.000625348')
('Epoch:', '0010', 'training time (minutes)=', '0.3127', 'loss=', '0.000575

In [1]:
Point_Cloud(ae.reconstruct(train_data.next_batch(1)[0])[0][0]).plot()

NameError: name 'Point_Cloud' is not defined

In [14]:
r0, r1, r2 = train_data.next_batch(500)
td = PointCloudDataSet(r0, labels=r1)
res = ae.evaluate(td, conf)
print res[1]

0.000503272912465
